In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
%config InlineBackend.figure_format='retina'

mpl.rcParams['savefig.dpi'] = 300
mpl.rcParams['font.size'] = 12

plt.style.use('tableau-colorblind10')

In [ ]:
# get file with nvidia-smi readings
url = "nvidia/nvidia_power_pretrain_16GB.csv"
df_orig = pd.read_csv(url)

In [ ]:
# isolate the training time
df_orig.timestamp = [datetime.strptime(time, "%Y/%m/%d %H:%M:%S.%f") for time in df_orig.timestamp]
df = df_orig.loc[df_orig[" utilization.gpu [%]"] > 0] #isolating training 

In [ ]:
# extract time for GPU, i.e., while GPU is used
time_interval = df.iloc[-1].timestamp - df.iloc[0].timestamp
total_time = time_interval.total_seconds()
print("Seconds: {}".format(total_time))
print ("Training hours: {}".format(total_time/3600))

In [ ]:
# Total time the program is run
time_interval_cpu = df_orig.iloc[-1].timestamp - df_orig.iloc[0].timestamp
total_time_cpu = time_interval_cpu.total_seconds()
print("Seconds: {}".format(total_time))
print ("Hours: {}".format(total_time/3600))

In [2]:
# GPU energy consumption
mean_power_draw =df[' power.draw [W]'].mean() 
gpu_energy = mean_power_draw * total_time 
sum_power_draw = df_orig[' power.draw [W]'].sum() # total power during the time code is running
gpu_util = df[' utilization.gpu [%]'].mean()
memory_util = df[' utilization.memory [%]'].mean()
memory_used = df[' memory.used [MiB]'].mean()
print('{} total from mean \n{} summed power'.format(gpu_energy,sum_power_draw))
print('in kWh ', sum_power_draw / (1000 * 3600))
print("GPU utilisation: ", gpu_util)

NameError: name 'df' is not defined

In [ ]:
df.plot(x= "timestamp",y = " power.draw [W]")

In [ ]:
print("GPU energy consumption: {} W".format(gpu_energy))
gpu_energy_kWh = gpu_energy / (1000 * 60 *60)
print("GPU energy consumption: {} kWh".format(gpu_energy_kWh))

In [ ]:
cpu_energy = 85 * total_time_cpu
total_energy = gpu_energy + cpu_energy
print (total_energy)
total_energy_kwh = total_energy / (1000 * 3600)
total_energy_kwh

## Power monitor readings

In [ ]:
print (df.iloc[-1].timestamp, df.iloc[0].timestamp)
print (df_orig.iloc[-1].timestamp, df_orig.iloc[0].timestamp)

In [ ]:
power_monitor_df = pd.read_csv("results/pretrain_power_monitor.csv")

In [ ]:
power_monitor_df['last_changed'] = pd.to_datetime(power_monitor_df['last_changed'], errors='coerce')
power_monitor_df.dropna(subset=['last_changed'], inplace=True)
power_monitor_df.set_index('last_changed', inplace=True)

In [ ]:
start_date = '2020-06-16'
end_date = '2020-06-16'
start_time = '21:38:59' #one hour behind
end_time = '23:18:51'
subset_df = power_monitor_df.loc[start_date:end_date].between_time(start_time, end_time)[['power_consumption']]
((subset_df['power_consumption']).sum() * 3) / (1000 * 3600) #sampled every 3 sec

In [ ]:
y_max = subset_df['power_consumption'].max() * 1.2
ax =subset_df[(power_monitor_df != 0).all(1)].plot(ylim=(0,y_max))
ax.set_title("Power monitor readings for BERT pretraining")
ax.set_ylabel('Power (W)')
ax.set_xlabel('Time')
plt.tight_layout()
ax.figure.savefig("viz/pm_BERT_pretrain_large.png")

In [ ]:
mean_power_monitor = subset_df["power_consumption"].mean()
mean_power_monitor